In [1]:
import requests

from bs4 import BeautifulSoup

In [2]:
r = requests.get('https://news.yandex.ru/politics.html')
soup = BeautifulSoup(r.text, 'html.parser')

urls_politics = soup.find_all('h2', class_='story__title')
urls_politics = ['https://news.yandex.ru' + t.find('a', href=True)['href'] for t in urls_politics]
urls_politics[:5]

['https://news.yandex.ru/story/Putin_reshenie_SSHA_o_vykhode_iz_DRSMD_ne_ostanetsya_bez_otveta--0eccc17d30741977028a7111240eec69?lr=213&lang=ru&stid=rgyFA-wa4oKjC1CrEd-1&rubric=politics&from=rubric',
 'https://news.yandex.ru/story/Ne_v_brov_a_v_gaz_Turciya_stanovitsya_khabom_dlya_rossijskogo_topliva--9d8b2db8ca20e33f05987babdbe95dd6?lr=213&lang=ru&stid=x2bMX0C26xYABi0MJTau&rubric=politics&from=rubric',
 'https://news.yandex.ru/story/Genprokuratura_Magnitskij_mog_byt_otravlen_v_interesakh_Braudera--7b16bb71c5e13422f68aaff39b183f29?lr=213&lang=ru&stid=_fLmSeauQdQYu5FWlbCt&rubric=politics&from=rubric',
 'https://news.yandex.ru/story/ES_poobeshhal_prinyat_celevye_mery_v_svyazi_s_situaciej_v_Azovskom_more--9a24f86cd9d584b5ec6a878fefb36696?lr=213&lang=ru&stid=5DWgE_2tbu9Fib62QP_o&rubric=politics&from=rubric',
 'https://news.yandex.ru/story/Rogozin_oznakomitsya_s_kosmicheskim_korablem_Ilona_Maska--2b908178e492dc401672e3d3db2ead71?lr=213&lang=ru&stid=e8uK_mkymmVDdichsSLJ&rubric=politics&from=r

In [3]:
len(urls_politics)

17

In [4]:
r = requests.get('https://news.yandex.ru/sport.html')
soup = BeautifulSoup(r.text, 'html.parser')

urls_sports = soup.find_all('h2', class_='story__title')
urls_sports = ['https://news.yandex.ru' + t.find('a', href=True)['href'] for t in urls_sports]
urls_sports[:5]

['https://news.yandex.ru/story/Kudryashov_pokinul_raspolozhenie_sbornoj_Rossii--53496ec6135b269b555e111342846c1b?lr=213&lang=ru&stid=-29tzJntsgnOzoMoQOsW&rubric=sport&from=rubric',
 'https://news.yandex.ru/story/Avangard_na_vyezde_pereigral_Torpedo--22a1dac57d6dabeb0109147d197bfe7d?lr=213&lang=ru&stid=EGfMYi5XOk343VvL_7Ci&rubric=sport&from=rubric',
 'https://news.yandex.ru/story/CSKA_obygral_Slovan_Kaprizov_oformil_khet-trik--cc9f553c4ee46925a65be3fed219f923?lr=213&lang=ru&stid=WWzJVHP6qbGjdkjrgMQE&rubric=sport&from=rubric',
 'https://news.yandex.ru/story/Napadayushhij_Kolambusa_EHtkinson_priznan_pervoj_zvezdoj_nedeli_v_NKHL--246f561c2d4ab2d7312e6ed7f0ed2466?lr=213&lang=ru&stid=lzQO-Yghmp1_du8DzpcR&rubric=sport&from=rubric',
 'https://news.yandex.ru/story/Luis_Suares_prokommentiroval_problemy_Dembele_v_Barselone--49864cda0c65a20b60b72c3ada94ead7?lr=213&lang=ru&stid=pbUvJTP0KHyxsZ8OQrCV&rubric=sport&from=rubric']

In [5]:
len(urls_sports)

17

In [6]:
import urllib
import os

def extract_texts(url, dir_):
    filename = urllib.parse.urlparse(url).path.split('/')[-1].split('-')[-1]
    filename = os.path.join(dir_, filename)
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    elem = soup.select_one('.doc__content')
    _ = elem.find(class_='doc__info').extract()
    text = elem.text
    
    with open(filename, mode='w') as f_text:
        f_text.write(text)

In [7]:
!mkdir -p data/politics data/sports

In [8]:
from functools import partial

from multiprocessing.pool import ThreadPool

with ThreadPool(8) as pool:
    extract_texts_politics = partial(extract_texts, dir_='./data/politics/')
    r = pool.map(extract_texts_politics, urls_politics)
    
with ThreadPool(8) as pool:
    extract_texts_sports = partial(extract_texts, dir_='./data/sports/')
    r = pool.map(extract_texts_sports, urls_sports)